In [10]:
import numpy as np
import cv2
from Undistort import Undistort
from Threshold import Threshold
from Warp import Warp
from Polyfit import Polyfit
from Polydrawer import Polydrawer
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from moviepy.video.io.VideoFileClip import VideoFileClip
from IPython.display import HTML

print('Loaded Modules!!')

Loaded Modules!!


In [2]:
undistort = Undistort()
polydraw = Polydrawer()
polyfit = Polyfit()

In [3]:
src = np.float32([
    [580, 460],
    [700, 460],
    [1040, 680],
    [260, 680],
])

dst = np.float32([
    [260, 0],
    [1040, 0],
    [1040, 720],
    [260, 720],
])

In [4]:
warp = Warp(src, dst)
undistort.calibratie_camera()

Working on Image :  ./camera_cal/calibration10.jpg
Working on Image :  ./camera_cal/calibration11.jpg
Working on Image :  ./camera_cal/calibration12.jpg
Working on Image :  ./camera_cal/calibration13.jpg
Working on Image :  ./camera_cal/calibration14.jpg
Working on Image :  ./camera_cal/calibration15.jpg
Working on Image :  ./camera_cal/calibration16.jpg
Working on Image :  ./camera_cal/calibration17.jpg
Working on Image :  ./camera_cal/calibration18.jpg
Working on Image :  ./camera_cal/calibration19.jpg
Working on Image :  ./camera_cal/calibration2.jpg
Working on Image :  ./camera_cal/calibration20.jpg
Working on Image :  ./camera_cal/calibration3.jpg
Working on Image :  ./camera_cal/calibration6.jpg
Working on Image :  ./camera_cal/calibration7.jpg
Working on Image :  ./camera_cal/calibration8.jpg
Working on Image :  ./camera_cal/calibration9.jpg


In [5]:
def process_images_test(base_image, name):

    img_undist = undistort.undistort(base_image)
    mpimg.imsave('./output_images/undistorted_{}'.format(name), img_undist)

    threshold = Threshold(kernel=3)
    img = threshold.combined_threshold(img_undist)
    mpimg.imsave('./output_images/thresholded_{}'.format(name), img, cmap="gray")

    img = warp.warp(img)
    mpimg.imsave('./output_images/warped_{}'.format(name), img, cmap="gray")

    left_fit, right_fit = polyfit.polyfit(img, visualize=True)

    img = polydraw.draw(img_undist, left_fit, right_fit, warp.Minv)

    mpimg.imsave('./output_images/final_{}'.format(name), img, cmap="gray")

In [6]:
test_images = ['straight_lines1.jpg', 'straight_lines2.jpg', 'test1.jpg', 'test2.jpg', 'test3.jpg', 'test4.jpg', 'test5.jpg', 'test6.jpg']
for image in test_images:
    img = mpimg.imread('./test_images/' + image)
    process_images_test(img, image)

In [7]:
def process_image(base_image):
    img_undist = undistort.undistort(base_image)
    threshold = Threshold(kernel=3)
    img = threshold.combined_threshold(img_undist)
    img = warp.warp(img)
    left_fit, right_fit = polyfit.polyfit(img, visualize=False)
    img = polydraw.draw(img_undist, left_fit, right_fit, warp.Minv)
    return img

In [12]:
white_output = 'project_video_done_2.mp4'.format(video)
clip1 = VideoFileClip('project_video.mp4')
white_clip = clip1.fl_image(process_image)  # NOTE: this function expects color images!!
white_clip.write_videofile(white_output, audio=False)

ProcessLookupError: [Errno 3] No such process

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))